# demo

> Demo of sample `ipyfernel` usage

In [ ]:
#| default_exp demo

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
from ipyfernel import * 

# Getting Started

In [ ]:
register_remote_kernel(remote_python='/Users/shawley/exercises/solveit/.venv/bin/python')

ipyf_remote_kernel is already a registered kernel


In [ ]:
port, user = 41192, 'shawley'
%set_remote {port} {user}

/app/data/.ssh/config file updated.


Success: remote kernel started


# Progress Bars

The remote execution has the same limitation that any other local solvit progress bars might have, e.g. nothing with `\r`.  Here's a simple one: 

In [ ]:
%%remote 
import time
import socket 
hostname = socket.gethostname()   # let's clarify that we're running remotely
print("We're executing on",hostname) 

def test_progbar():
    print("[", end="", flush=True)
    for i in range(20):
        print("=", end="", flush=True)
        time.sleep(0.15)
    print("] Done!")

test_progbar()

We're executing on Chonk
[

=

=

=

=

...

=

=

=

=

] Done!


Chonk is the name of my remote laptop, and that progress bar went across the screen in real time over the span of a few seconds.